# 3b. Enrich Stream
  --------------------------------------------------------------------

Receive a stream of events from from `incoming-events-stream` and enrichthe relevant events with socioeconomic data by looking up the enrichment table. The enriched data is streamed out to `enriched-events-stream`.

![Model deployment with streaming Real-time operational Pipeline](../../assets/images/model-deployment-with-streaming.png)

## Create and Test a Local Function 
Import nuclio SDK and magics, <b>do not remove the cell and comment !!!</b>

In [ ]:
# nuclio: ignore
import nuclio

#### Functions imports

In [ ]:
# nuclio: start-code

In [ ]:
%load config.py

In [ ]:
%load env.py

In [ ]:
import os
import hashlib
import json
import v3io.dataplane

<b>Specify function dependencies and configuration<b>

In [ ]:
%nuclio cmd -c pip install v3io

In [ ]:
%%nuclio env
V3IO_ACCESS_KEY = ${V3IO_ACCESS_KEY}
CONTAINER = users
MDWS_ES_OUTPUT_STREAM_PATH = ${MDWS_ES_OUTPUT_STREAM_PATH}
PARTITION_ATTR = ${PARTITION_ATTR}

MDWS_ENRICHMENT_TABLE_PATH = ${MDWS_ENRICHMENT_TABLE_PATH}
ENRICHMENT_KEY = postcode

In [ ]:
%%nuclio config
spec.triggers.v3io_stream.kind = "v3ioStream"
spec.triggers.v3io_stream.disabled = false
spec.triggers.v3io_stream.url = "${MDWS_ES_INPUT_OUTPUT_STREAM_PATH}"
spec.triggers.v3io_stream.maxWorkers = 10
spec.triggers.v3io_stream.password = "${V3IO_ACCESS_KEY}"
spec.triggers.v3io_stream.attributes.pollingIntervalMs = 500
spec.triggers.v3io_stream.attributes.seekTo = "earliest"
spec.triggers.v3io_stream.attributes.readBatchSize = 64


## Function code

In [ ]:
def init_context(context):
    V3IO_ACCESS_KEY = os.getenv('V3IO_ACCESS_KEY')
    CONTAINER = os.getenv('CONTAINER')
    OUTPUT_STREAM_PATH = os.getenv('MDWS_ES_OUTPUT_STREAM_PATH')
    PARTITION_ATTR = os.getenv('PARTITION_ATTR')
    ENRICHMENT_TABLE_PATH = os.getenv('MDWS_ENRICHMENT_TABLE_PATH')
    ENRICHMENT_KEY = os.getenv('ENRICHMENT_KEY')
    v3io_client = v3io.dataplane.Client(endpoint='http://v3io-webapi:8081', access_key=V3IO_ACCESS_KEY)
    
    setattr(context, 'v3io_client', v3io_client)
    setattr(context, 'partition_attr', PARTITION_ATTR)
    setattr(context, 'container', CONTAINER)
    setattr(context, 'output_stream_path', OUTPUT_STREAM_PATH)
    
    setattr(context, 'enrichment_table_path', ENRICHMENT_TABLE_PATH)
    setattr(context, 'enrichment_key', ENRICHMENT_KEY)


def handler(context, event):
    if type(event.body) is dict:
        event_dict = event.body
    else:
        event_dict = json.loads(event.body)
        
    context.logger.info_with('Got invoked',
                             trigger_kind=event.trigger.kind,
                             event_body=event_dict)
        
    partition_key = event_dict.get(context.partition_attr)
    
    record = {}
    if event_dict['event_type'] == 'registration':
        enriched_event = enrich_event(context, event_dict)
        record = event_to_record(enriched_event, partition_key)
    else:
        record = event_to_record(event_dict, partition_key)
    print("dsadsadad" + context.output_stream_path)
    resp = context.v3io_client.put_records(container=context.container, 
                                   path=context.output_stream_path, 
                                   records=[record], 
                                   raise_for_status=v3io.dataplane.RaiseForStatus.never)
    
    context.logger.info_with('Sent event to stream', 
                             record=record,
                             response_status=resp.status_code, 
                             response_body=resp.body.decode('utf-8'))
    
    return resp.status_code


def enrich_event(context, event_dict):
    if context.enrichment_key in event_dict:
        enrichment_key_value = event_dict[context.enrichment_key]
        resp = context.v3io_client.get_item(container=context.container, 
                                            path=os.path.join(context.enrichment_table_path, str(enrichment_key_value)),
                                           raise_for_status=v3io.dataplane.RaiseForStatus.never)
        if 200 <= resp.status_code <= 299:
            enriched_event = {**event_dict, **resp.output.item}
            context.logger.info_with('Event was enriched', enriched_event=enriched_event)
            return enriched_event
        else:
            context.logger.debug_with("Couldn't enrich event", 
                                      enrichment_key_value=enrichment_key_value,
                                      response_status=resp.status_code, 
                                      response_body=resp.body.decode('utf-8'))
            return event_dict
    else:
        return event_dict

    
def event_to_record(event_dict, partition_key):
    event_str = json.dumps(event_dict)
    return {'data': event_str, 'partition_key': str(partition_key)}

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [ ]:
# nuclio: end-code
# marks the end of a code section

## Test locally

In [ ]:
event = nuclio.Event(body=b'{"user_id" : 111111 , "event_type": "registration", "postcode": 11014}')
init_context(context)
handler(context, event)


## Deploy function

In [ ]:
%nuclio deploy -p ${MDWS_PROJECT_NAME} -n ${V3IO_USERNAME}-enrich-stream